# Feature Backfill

explanation here

### Load imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import hopsworks
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

#### Helper functions

In [2]:
# functions for replacing date and time with timestamp (seconds since 1970-01-01)

def energy_timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%d.%m.%Y %H:%M')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

def weather_timestamp_2_time(x):
    dt_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    dt_obj = dt_obj.timestamp() * 1000

    return int(dt_obj)

#### Load all datasets

In [39]:
day_ahead_prices = pd.read_csv("data/Day-ahead Prices_202201010000-202301010000.csv")
gbg = pd.read_csv("data/smhi-opendata_2022_goteborg_temperatur.csv")
act_gen_per_prodtype = pd.read_csv("data/Actual Generation per Production Type_202201010000-202301010000.csv")
total_load = pd.read_csv("data/Total Load - Day Ahead _ Actual_202201010000-202301010000.csv")

## Data preparation

### Total load (consumption)

In [40]:
total_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 3 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Time (CET/CEST)                               8761 non-null   object 
 1   Day-ahead Total Load Forecast [MW] - BZN|SE3  8760 non-null   float64
 2   Actual Total Load [MW] - BZN|SE3              8760 non-null   float64
dtypes: float64(2), object(1)
memory usage: 205.5+ KB


In [41]:
total_load[['DateTime', 'dt_end']] = total_load['Time (CET/CEST)'].str.split(' - ', expand = True)
#total_load[['date', 'time']] = total_load['DateTime'].str.split(' ', expand = True)
total_load.drop(['dt_end','Time (CET/CEST)','Day-ahead Total Load Forecast [MW] - BZN|SE3'],axis=1, inplace=True)
total_load['DateTime'] = pd.to_datetime(total_load['DateTime'], infer_datetime_format=True)
#total_load['DateTime'] = total_load['DateTime'].apply(energy_timestamp_2_time)
total_load.head()

,Actual Total Load [MW] - BZN|SE3,DateTime
0,9685.0,2022-01-01 00:00:00
1,9612.0,2022-01-01 01:00:00
2,9601.0,2022-01-01 02:00:00
3,9552.0,2022-01-01 03:00:00
4,9449.0,2022-01-01 04:00:00


### Day-price data

In [42]:
day_ahead_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MTU (CET/CEST)             8761 non-null   object 
 1   Day-ahead Price [EUR/MWh]  8760 non-null   float64
 2   Currency                   8760 non-null   object 
 3   BZN|SE3                    0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 273.9+ KB


In [43]:
day_ahead_prices[['DateTime', 'dt_end']] = day_ahead_prices['MTU (CET/CEST)'].str.split(' - ', expand = True)
#day_ahead_prices[['date', 'time']] = day_ahead_prices['DateTime'].str.split(' ', expand = True)
day_ahead_prices.drop(['dt_end','MTU (CET/CEST)', 'Currency', 'BZN|SE3'],axis=1, inplace=True)
#day_ahead_prices['DateTime'] = day_ahead_prices['DateTime'].apply(energy_timestamp_2_time)
day_ahead_prices['DateTime'] = pd.to_datetime(day_ahead_prices['DateTime'], infer_datetime_format=True)
day_ahead_prices.head()

,Day-ahead Price [EUR/MWh],DateTime
0,46.60,2022-01-01 00:00:00
1,41.33,2022-01-01 01:00:00
2,42.18,2022-01-01 02:00:00
3,44.37,2022-01-01 03:00:00
4,37.67,2022-01-01 04:00:00


### Electricity data

In [44]:
act_gen_per_prodtype.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Area                                             8761 non-null   object 
 1   MTU                                              8761 non-null   object 
 2   Fossil Gas  - Actual Aggregated [MW]             8759 non-null   float64
 3   Hydro Water Reservoir  - Actual Aggregated [MW]  8759 non-null   float64
 4   Nuclear  - Actual Aggregated [MW]                8759 non-null   float64
 5   Other  - Actual Aggregated [MW]                  8759 non-null   float64
 6   Solar  - Actual Aggregated [MW]                  8759 non-null   float64
 7   Wind Onshore  - Actual Aggregated [MW]           8759 non-null   float64
dtypes: float64(6), object(2)
memory usage: 547.7+ KB


In [45]:
## remove non-informative or weak columns
act_gen_per_prodtype[['DateTime', 'dt_end']] = act_gen_per_prodtype['MTU'].str.split(' - ', expand = True)
#act_gen_per_prodtype[['date', 'time']] = act_gen_per_prodtype['DateTime'].str.split(' ', expand = True)
act_gen_per_prodtype = act_gen_per_prodtype.drop(['dt_end', 'MTU', 'Area', 'Fossil Gas  - Actual Aggregated [MW]'], axis=1)
act_gen_per_prodtype.rename(columns={
                                "Hydro Water Reservoir  - Actual Aggregated [MW]": "hydro_water_reservoir",
                                "Nuclear  - Actual Aggregated [MW]": "nuclear",
                                "Other  - Actual Aggregated [MW]": "other",
                                "Solar  - Actual Aggregated [MW]": "solar",
                                "Wind Onshore  - Actual Aggregated [MW]": "wind_onshore"}, inplace=True)
#act_gen_per_prodtype['DateTime'] = act_gen_per_prodtype['DateTime'].apply(energy_timestamp_2_time)   
act_gen_per_prodtype['DateTime'] = pd.to_datetime(act_gen_per_prodtype['DateTime'], infer_datetime_format=True)
act_gen_per_prodtype.head(3)


,hydro_water_reservoir,nuclear,other,solar,wind_onshore,DateTime
0,824.0,5841.0,892.0,0.0,1062.0,2022-01-01 00:00:00
1,818.0,5842.0,851.0,0.0,999.0,2022-01-01 01:00:00
2,818.0,5839.0,858.0,0.0,958.0,2022-01-01 02:00:00


### Weather temperature

In [46]:
gbg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8755 entries, 0 to 8754
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Datum           8755 non-null   object 
 1   Tid (UTC)       8755 non-null   object 
 2   Lufttemperatur  8755 non-null   float64
 3   Kvalitet        8755 non-null   object 
dtypes: float64(1), object(3)
memory usage: 273.7+ KB


In [47]:
gbg_temp = gbg.copy()
gbg_temp['DateTime'] = gbg_temp.agg('{0[Datum]} {0[Tid (UTC)]}'.format, axis=1)
gbg_temp['DateTime'] = pd.to_datetime(gbg_temp['DateTime'], infer_datetime_format=True)
# #gbg_temp['DateTime'] = gbg_temp['DateTime'].apply(weather_timestamp_2_time) 
#gbg_temp['Datum'] = pd.to_datetime(gbg_temp['Datum'], infer_datetime_format=True)
#gbg_temp['Tid (UTC)'] = pd.to_datetime(gbg_temp['Tid (UTC)'], infer_datetime_format=True)
gbg_temp = gbg_temp.drop(['Kvalitet', 'Datum', 'Tid (UTC)'],axis=1)
gbg_temp.head()

,Lufttemperatur,DateTime
0,5.3,2022-01-01 00:00:00
1,5.4,2022-01-01 01:00:00
2,5.4,2022-01-01 02:00:00
3,5.7,2022-01-01 03:00:00
4,5.3,2022-01-01 04:00:00


### Combine datasets

In [48]:
act_gen_per_prodtype_price = act_gen_per_prodtype.merge(day_ahead_prices, how='inner', on='DateTime')
act_gen_per_prodtype_price = act_gen_per_prodtype_price.merge(total_load, how='inner', on='DateTime')
act_gen_per_prodtype_price = act_gen_per_prodtype_price.merge(gbg_temp, how='inner', on='DateTime')
act_gen_per_prodtype_price.head()

,hydro_water_reservoir,nuclear,other,solar,wind_onshore,DateTime,Day-ahead Price [EUR/MWh],Actual Total Load [MW] - BZN|SE3,Lufttemperatur
0,824.0,5841.0,892.0,0.0,1062.0,2022-01-01 00:00:00,46.60,9685.0,5.3
1,818.0,5842.0,851.0,0.0,999.0,2022-01-01 01:00:00,41.33,9612.0,5.4
2,818.0,5839.0,858.0,0.0,958.0,2022-01-01 02:00:00,42.18,9601.0,5.4
3,838.0,5839.0,859.0,0.0,888.0,2022-01-01 03:00:00,44.37,9552.0,5.7
4,842.0,5840.0,856.0,0.0,788.0,2022-01-01 04:00:00,37.67,9449.0,5.3


In [49]:
# final adjustments
act_gen_per_prodtype_price.rename(columns={"Day-ahead Price [EUR/MWh]" : "day_ahead_price", 
                                        "Actual Total Load [MW] - BZN|SE3" : "total_load" , 
                                        "Lufttemperatur" : "temperature"}, inplace=True)
act_gen_per_prodtype_price.head()

,hydro_water_reservoir,nuclear,other,solar,wind_onshore,DateTime,day_ahead_price,total_load,temperature
0,824.0,5841.0,892.0,0.0,1062.0,2022-01-01 00:00:00,46.60,9685.0,5.3
1,818.0,5842.0,851.0,0.0,999.0,2022-01-01 01:00:00,41.33,9612.0,5.4
2,818.0,5839.0,858.0,0.0,958.0,2022-01-01 02:00:00,42.18,9601.0,5.4
3,838.0,5839.0,859.0,0.0,888.0,2022-01-01 03:00:00,44.37,9552.0,5.7
4,842.0,5840.0,856.0,0.0,788.0,2022-01-01 04:00:00,37.67,9449.0,5.3


### Check for Nan or zeroes

In [50]:
print('There are {} missing values or NaNs in df_energy.'
      .format(act_gen_per_prodtype_price.isnull().values.sum()))

temp_energy = act_gen_per_prodtype_price.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_energy based on all columns.'
      .format(temp_energy))

There are 12 missing values or NaNs in df_energy.
There are 0 duplicate rows in df_energy based on all columns.


In [51]:
# check for duplicate rows
act_gen_per_prodtype_price.loc[act_gen_per_prodtype_price.duplicated]

,hydro_water_reservoir,nuclear,other,solar,wind_onshore,DateTime,day_ahead_price,total_load,temperature


In [52]:
# check for null/undef values
act_gen_per_prodtype_price.loc[act_gen_per_prodtype_price.isna().any(axis=1)]

,hydro_water_reservoir,nuclear,other,solar,wind_onshore,DateTime,day_ahead_price,total_load,temperature
2042,NaN,NaN,NaN,NaN,NaN,2022-03-27 02:00:00,NaN,NaN,3.8
2063,NaN,NaN,NaN,NaN,NaN,2022-03-27 23:00:00,11.6,9190.0,6.9


In [53]:
# Since its only a few rows, use linear interpolation on each column that contains nan/null values.
col = act_gen_per_prodtype_price.columns[act_gen_per_prodtype_price.isna().any()]
for i in col:
    act_gen_per_prodtype_price[i].interpolate(method='linear', limit_direction='forward', axis=0, inplace=True)
    
# Also remove duplicate rows
act_gen_per_prodtype_price.drop_duplicates(inplace=True)


In [38]:
# Final check
print('There are {} missing values or NaNs in df_energy.'
      .format(act_gen_per_prodtype_price.isnull().values.sum()))

temp_energy = act_gen_per_prodtype_price.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_energy based on all columns.'
      .format(temp_energy))
act_gen_per_prodtype_price.info()

There are 0 missing values or NaNs in df_energy.
There are 0 duplicate rows in df_energy based on all columns.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8762 entries, 0 to 8761
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   hydro_water_reservoir  8762 non-null   float64       
 1   nuclear                8762 non-null   float64       
 2   other                  8762 non-null   float64       
 3   solar                  8762 non-null   float64       
 4   wind_onshore           8762 non-null   float64       
 5   DateTime               8762 non-null   datetime64[ns]
 6   day_ahead_price        8762 non-null   float64       
 7   total_load             8762 non-null   float64       
 8   temperature            8762 non-null   float64       
dtypes: datetime64[ns](1), float64(8)
memory usage: 684.5 KB


In [76]:
act_gen_per_prodtype_price['DateTime'] = act_gen_per_prodtype_price['DateTime'].apply(weather_timestamp_2_time)
act_gen_per_prodtype_price.head(7)

,hydro_water_reservoir,nuclear,other,solar,wind_onshore,DateTime,day_ahead_price,total_load,temperature
0,824.0,5841.0,892.0,0.0,1062.0,1640991600000,46.60,9685.0,5.3
1,818.0,5842.0,851.0,0.0,999.0,1640995200000,41.33,9612.0,5.4
2,818.0,5839.0,858.0,0.0,958.0,1640998800000,42.18,9601.0,5.4
3,838.0,5839.0,859.0,0.0,888.0,1641002400000,44.37,9552.0,5.7
4,842.0,5840.0,856.0,0.0,788.0,1641006000000,37.67,9449.0,5.3
5,841.0,5839.0,880.0,0.0,734.0,1641009600000,39.70,9614.0,4.4
6,868.0,5839.0,913.0,0.0,646.0,1641013200000,40.59,9920.0,4.7


## Create feature groups on Hopsworks

In [78]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/4247
Connected. Call `.close()` to terminate connection gracefully.


In [80]:
new_electricity_data_fg = fs.get_or_create_feature_group(
        name = 'new_electricity_data_fg', ## UPDATED
        description = 'Dataset of Electricity production, consumption and price in SE3 area, with hourly weather temperature',
        version = 1,
        primary_key = ['DateTime'],
    )    

new_electricity_data_fg.insert(act_gen_per_prodtype_price, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/4247/fs/4192/fg/14993


Uploading Dataframe: 100.00% |██████████| Rows 8762/8762 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/4247/jobs/named/new_electricity_data_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f017d672130>, None)

%6|1673612056.721|FAIL|Abyel-PC#producer-2| [thrd:ssl://52.14.58.146:9092/bootstrap]: ssl://52.14.58.146:9092/1: Disconnected (after 3881134ms in state UP)
%6|1673612200.872|FAIL|Abyel-PC#producer-1| [thrd:ssl://3.138.67.216:9092/bootstrap]: ssl://3.138.67.216:9092/2: Disconnected (after 3899384ms in state UP, 1 identical error(s) suppressed)
